<a id="top"></a>

# 02 · Data Smoke & Bench

**Qué hace este notebook:**

Pruebas rápidas para validar **datos/splits**, comprobar el **pipeline de codificación temporal** (offline H5 o runtime) y hacer **micro-bench** de *throughput*. Lee la configuración desde `configs/presets.yaml` para asegurar coherencia con el cuaderno 03.

---

## 🎯 Objetivos
- Detectar rápido problemas en datos/H5
- Tener una referencia de it/s y formas de tensores antes de lanzar entrenos largos.

<a id="toc"></a>

## 🧭 Índice

- [1) Setup del entorno e imports](#sec-01)
- [2) Cargar preset y eco de configuración](#sec-02)
- [3) Verificación de datos y carga de `task_list`](#sec-03)
- [4) Factory de DataLoaders (H5 offline o CSV + runtime)](#sec-04)
- [5) Smoke universal (loader → modelo)](#sec-05)
- [6) Forward manual con fallback AMP](#sec-06)
- [7) Echo de configuración del bench](#sec-07)
- [8) Toggle: *it/s* por época](#sec-08)
- [9) (Opc.) Micro-bench pipeline *it/s*](#sec-09)
- [10) (Opc.) Smokes de métodos con `run_continual`](#sec-10)


<a id="sec-01"></a>
## 1) Setup del entorno e imports

**Objetivo:** dejar preparado el entorno para realizar *smoke tests* (comprobaciones mínimas de punta a punta) y micro-*benchmarks* de **loader → forward** sin entrenar.

Esta celda:
- Limita hilos BLAS (`OMP`, `MKL`, `OPENBLAS`) para evitar *over-subscription*.
- Detecta la raíz del repo (`ROOT`) y la añade a `sys.path`.
- Importa utilidades de *bench* y helpers del proyecto:
  - **Datos/loader**: `build_make_loader_fn`, `ImageTransform`, `AugmentConfig`.
  - **Modelo**: `build_model`.
  - **Entrenamiento**: `set_encode_runtime` y utilidades de `training`.
  - **Bench**: `universal_smoke_forward`, `enable_epoch_ips`, `disable_epoch_ips`, `print_bench_config`, `pipeline_its`.
- Selecciona dispositivo (`cuda` si está disponible) y activa optimizaciones razonables:
  - `cudnn.benchmark`, TF32 en CUDA y precisión alta en *matmul*.

> **Nota:** Este cuaderno **no entrena**. Su meta es validar que tu configuración de datos, *loader* y modelo **encienden** y medir un *throughput* de referencia.

[↑ Volver al índice](#toc)


In [1]:
# =============================================================================
# Imports y setup
# =============================================================================
import os
os.environ["OMP_NUM_THREADS"] = "1"
os.environ["MKL_NUM_THREADS"] = "1"
os.environ["OPENBLAS_NUM_THREADS"] = "1"

from pathlib import Path
import sys, json, torch

ROOT = Path.cwd().parents[0] if (Path.cwd().name == "notebooks") else Path.cwd()
if str(ROOT) not in sys.path:
    sys.path.append(str(ROOT))

from src.config import load_preset, set_seeds, build_make_loader_fn  # ← nuevo
from src.datasets import ImageTransform, AugmentConfig
from src.models import build_model
import src.training as training
from src.bench import (
    universal_smoke_forward,
    enable_epoch_ips, disable_epoch_ips,
    print_bench_config,
    pipeline_its,
    to_5d,  # ← lo usaremos en la celda 6 para evitar duplicar lógica
)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
torch.set_num_threads(4)
torch.backends.cudnn.benchmark = True
torch.backends.cuda.matmul.allow_tf32 = True
torch.backends.cudnn.allow_tf32 = True
try:
    torch.set_float32_matmul_precision("high")
except Exception:
    pass

print("Device:", device)


Device: cuda


<a id="sec-02"></a>
## 2) Config desde `presets.yaml` y eco de configuración

**Objetivo:** usar exactamente la **misma configuración** que en entrenamiento (03), leyendo el `PRESET` de `configs/presets.yaml`.

Incluye:
- **Modelo y *transform*** (tamaño y escala a gris) vía `ImageTransform`.
- **Codificación temporal**: `ENCODER` (`rate`/`latency`), `T`, `GAIN` y `SEED`.
- **Modo de datos**:
  - `use_offline_spikes`: si es `true`, esperamos **H5** pre-generados.
  - `encode_runtime`: si es `true`, el *loader* entregará imágenes y se codificarán a eventos **en GPU**.
- **DataLoader**: `num_workers`, `prefetch_factor`, `pin_memory`, `persistent_workers`.
- **Augment** (train) y **balanceo online** (si lo activas).
- Define `make_model_fn(tfm)` para instanciar el modelo con la misma *transform*.

> **Consejo:** pon todos los cambios (modelo, codificación, *augment*, *loader*) en el **YAML**, así 02/03/04 usan la misma fuente de verdad.

[↑ Volver al índice](#toc)


In [ ]:
# =============================================================================
# Config: lee presets.yaml
# =============================================================================
PRESET = "fast"  # fast | std | accurate
CFG = load_preset(ROOT / "configs" / "presets.yaml", PRESET)

# Modelo / tfm
MODEL_NAME = CFG["model"]["name"]
tfm = ImageTransform(
    CFG["model"]["img_w"], CFG["model"]["img_h"],
    to_gray=bool(CFG["model"]["to_gray"]),
    crop_top=CFG["model"].get("crop_top", 0), crop_bottom=CFG["model"].get("crop_bottom", 0),
)

# Datos / codificación temporal
ENCODER = CFG["data"]["encoder"]
T       = int(CFG["data"]["T"])
GAIN    = float(CFG["data"]["gain"])
SEED    = int(CFG["data"]["seed"])

USE_OFFLINE_SPIKES   = bool(CFG["data"].get("use_offline_spikes", False))
RUNTIME_ENCODE       = bool(CFG["data"].get("encode_runtime", False))

# ---- DataLoader / augment / balanceo ----------------------------------------
NUM_WORKERS = int(CFG["data"].get("num_workers") or 0)
PREFETCH    = CFG["data"].get("prefetch_factor")
PIN_MEMORY  = bool(CFG["data"].get("pin_memory", True))
PERSISTENT  = bool(CFG["data"].get("persistent_workers", True))

AUG_CFG = AugmentConfig(**(CFG["data"].get("aug_train") or {})) \
          if CFG["data"].get("aug_train") else None

USE_ONLINE_BALANCING = bool(CFG["data"].get("balance_online", False))
BAL_BINS = int(CFG["data"].get("balance_bins") or 50)
BAL_EPS  = float(CFG["data"].get("balance_smooth_eps") or 1e-3)

print(f"[PRESET={PRESET}] model={MODEL_NAME} {tfm.w}x{tfm.h} gray={tfm.to_gray}")
print(f"[DATA] encoder={ENCODER} T={T} gain={GAIN} seed={SEED}")
print(f"[LOADER] workers={NUM_WORKERS} prefetch={PREFETCH} pin={PIN_MEMORY} persistent={PERSISTENT}")
print(f"[BALANCE] online={USE_ONLINE_BALANCING} bins={BAL_BINS}")
print(f"[RUNTIME_ENCODE] {RUNTIME_ENCODE} | [OFFLINE_SPIKES] {USE_OFFLINE_SPIKES}")

def make_model_fn(tfm):
    # kwargs solo necesarios para pilotnet_snn
    return build_model(MODEL_NAME, tfm, beta=0.9, threshold=0.5)


[PRESET=fast] model=pilotnet_snn 200x66 gray=True
[DATA] encoder=rate T=10 gain=0.5 seed=42
[LOADER] workers=8 prefetch=2 pin=True persistent=True
[BALANCE] online=False bins=50
[RUNTIME_ENCODE] False | [OFFLINE_SPIKES] True


<a id="sec-03"></a>
## 3) Verificación de datos y carga de `task_list`

Esta celda:
1. **Elige el fichero de tareas** en función del preset:
   - Si `prep.use_balanced_tasks = true` y existe, usa `tasks_balanced.json`.
   - En caso contrario, usa `tasks.json`.
2. Construye `task_list` con cada *run* y las rutas de `train/val/test`.
3. **Guardarraíl** (si `use_balanced_tasks = true`): exige que `train` apunte a `train_balanced.csv`.
4. Verifica según el **modo de datos**:
   - **H5 offline** (`use_offline_spikes = true`): comprueba que existen los H5 esperados para cada *split* con el **naming** coherente (`{split}_{encoder}_T{T}_gain{...}_{gray|rgb}_{W}x{H}.h5`).
   - **CSV + runtime**: valida que los CSV existen y que las rutas de imagen son reales (muestra mínima). Si esperabas material en `aug/` y no está, re-genera con `01A_PREP_BALANCED`.

> Si faltan H5 y estás en modo offline, **généralos** con `02_ENCODE_OFFLINE.ipynb`.  
> Si estás en modo runtime, asegúrate de que las rutas de imagen del CSV son absolutas o resolubles desde `ROOT`.

[↑ Volver al índice](#toc)


In [3]:
# =============================================================================
# Verificación de datos + selección de tasks según el PRESET
# =============================================================================
from pathlib import Path as _P

PROC = ROOT / "data" / "processed"
RAW  = ROOT / "data" / "raw" / "udacity"

# --- Elegir tasks según el preset ---
tb_name = CFG["prep"]["tasks_balanced_file_name"]   # p.ej., "tasks_balanced.json"
t_name  = CFG["prep"]["tasks_file_name"]            # p.ej., "tasks.json"
USE_BALANCED = bool(CFG.get("prep", {}).get("use_balanced_tasks", False))

cand_bal = PROC / tb_name
cand_std = PROC / t_name

if USE_BALANCED and cand_bal.exists():
    TASKS_FILE = cand_bal
else:
    TASKS_FILE = cand_std

with open(TASKS_FILE, "r", encoding="utf-8") as f:
    tasks_json = json.load(f)

task_list = [{"name": n, "paths": tasks_json["splits"][n]} for n in tasks_json["tasks_order"]]
print("Usando tasks:", TASKS_FILE.name)
for t in task_list:
    print(f" - {t['name']}: {_P(t['paths']['train']).name}")

# --- Guardarraíl: si usamos tasks balanceadas, aseguramos train_balanced.csv ---
if USE_BALANCED:
    for t in task_list:
        train_path = _P(t["paths"]["train"])
        if train_path.name != "train_balanced.csv":
            raise RuntimeError(
                f"[{t['name']}] Esperaba 'train_balanced.csv' en tasks balanceadas, "
                f"pero encontré '{train_path.name}'."
            )
    print("Verificación de tasks balanceadas OK.")

# --- Verificación específica según el modo de datos ---
if USE_OFFLINE_SPIKES:
    # Comprobamos que existen los H5 requeridos por el preset
    mw, mh = CFG["model"]["img_w"], CFG["model"]["img_h"]
    color = "gray" if CFG["model"]["to_gray"] else "rgb"
    gain_tag = (GAIN if ENCODER == "rate" else 0)
    missing = []

    for t in task_list:
        run = t["name"]
        base = PROC / run
        for split in ("train", "val", "test"):
            expected = base / f"{split}_{ENCODER}_T{T}_gain{gain_tag}_{color}_{mw}x{mh}.h5"
            if not expected.exists():
                missing.append(str(expected))

    if missing:
        raise FileNotFoundError(
            "Faltan H5 requeridos por el preset. Genera primero con 02_ENCODE_OFFLINE.ipynb.\n"
            + "\n".join(" - " + m for m in missing)
        )
    print("OK: H5 requeridos encontrados para todos los splits.")

else:
    # CSV + encode en runtime: comprobamos que el CSV existe y que hay imágenes accesibles
    import pandas as _pd

    for t in task_list:
        for split in ("train", "val", "test"):
            csv_path = _P(tasks_json["splits"][t["name"]][split])
            if not csv_path.is_absolute():
                csv_path = ROOT / csv_path
            if not csv_path.exists():
                raise FileNotFoundError(f"No existe CSV: {csv_path}")

    # Muestra mínima para detectar rutas rotas (ej. 'aug/' sin materializar)
    run0 = task_list[0]["name"]
    csv0 = _P(tasks_json["splits"][run0]["train"])
    if not csv0.is_absolute():
        csv0 = ROOT / csv0
    df0 = _pd.read_csv(csv0, nrows=5)
    col_path = "path" if "path" in df0.columns else None
    if col_path is None:
        # fallback: si el CSV tiene 'center/left/right', tomamos 'center'
        for c in ("center", "image", "img"):
            if c in df0.columns:
                col_path = c
                break
    if col_path is None:
        raise RuntimeError(
            f"No encuentro columna de ruta en {csv0}. Esperaba 'path' o 'center'."
        )

    missing_imgs = [p for p in df0[col_path].tolist() if not _P(p).exists()]
    if missing_imgs:
        raise FileNotFoundError(
            "Rutas de imagen no válidas en CSV (ejemplo):\n"
            f" - {missing_imgs[0]}\n"
            "Si esperabas imágenes balanceadas en 'aug/', re-ejecuta 01A_PREP_BALANCED.ipynb."
        )
    print("OK: CSV y rutas de imágenes accesibles para encode en runtime.")


Usando tasks: tasks_balanced.json
 - circuito1: train_balanced.csv
 - circuito2: train_balanced.csv
Verificación de tasks balanceadas OK.
OK: H5 requeridos encontrados para todos los splits.


<a id="sec-04"></a>
## 4) Factory de DataLoaders (H5 offline o CSV + runtime)

Crea `make_loader_fn` con los *flags* del preset:
- Si `use_offline_spikes = true`, prioriza **H5**.
- Si no hay H5 o `use_offline_spikes = false`, usa **CSV + codificación en runtime** (en GPU si está disponible).
- Aplica por defecto los *kwargs* del `DataLoader` (workers, *prefetch*, *pin/persistent*), *augment* de `train` y **balanceo online** si lo activas.

`make_loader_fn(...)` permite **sobrescribir** puntualmente *kwargs* (p. ej., bajar `batch_size` para un *smoke*).

[↑ Volver al índice](#toc)


In [4]:
# =============================================================================
# Factory de loaders (H5 offline o CSV + runtime encode)
#   Igual que en 03_TRAIN_CONTINUAL: elige H5 si use_offline_spikes=True,
#   y si no hay H5 cae a CSV + encode en GPU.
# =============================================================================
_raw_make_loader_fn = build_make_loader_fn(
    root=ROOT,
    use_offline_spikes=USE_OFFLINE_SPIKES,
    encode_runtime=RUNTIME_ENCODE,
)

# kwargs comunes del DataLoader que queremos aplicar por defecto
_DL_KW = dict(
    num_workers=NUM_WORKERS,
    prefetch_factor=PREFETCH,
    pin_memory=PIN_MEMORY,
    persistent_workers=PERSISTENT,
    aug_train=AUG_CFG,
    balance_train=USE_ONLINE_BALANCING,
    balance_bins=BAL_BINS,
    balance_smooth_eps=BAL_EPS,
)

def make_loader_fn(task, batch_size, encoder, T, gain, tfm, seed, **dl_kwargs):
    """Wrapper pass-through (permite override puntual de kwargs si se pasan)."""
    return _raw_make_loader_fn(
        task=task, batch_size=batch_size, encoder=encoder, T=T, gain=gain,
        tfm=tfm, seed=seed, **{**_DL_KW, **dl_kwargs}
    )

print("make_loader_fn listo (H5 si use_offline_spikes=True; fallback CSV+runtime si no).")


make_loader_fn listo (H5 si use_offline_spikes=True; fallback CSV+runtime si no).


<a id="sec-05"></a>
## 5) Smoke universal (loader → modelo)

Lanza un *smoke test* con el **primer `task`**:
- Inicializa un *loader* coherente con el preset (H5 o CSV+runtime).
- Comprueba que produce *batches* con la forma esperada.
- Ejecuta un **forward** del modelo (con AMP en CUDA si procede).
- Reporta errores tempranos de forma clara (shapes, *dtype*, dispositivo).

> Útil para detectar rápido desajustes entre `ENCODER`/`T`/`GAIN`, la forma del *batch* y lo que espera el modelo.

[↑ Volver al índice](#toc)


In [5]:
# =============================================================================
# Smoke universal de loader+modelo (usa variables del preset)
# =============================================================================
_ = universal_smoke_forward(
    make_loader_fn,
    task=task_list[0],
    encoder=ENCODER, T=T, gain=GAIN,
    tfm=tfm, seed=SEED, device=device,
    use_encode_runtime=RUNTIME_ENCODE,
)


dataset ya codificado; solo permuto a (T,B,C,H,W)
x5d.device: cpu | shape: (10, 8, 1, 66, 200)
[forward] ejecutado con AMP


<a id="sec-06"></a>
## 6) Forward manual con fallback AMP

Ejecuta el camino completo de forma explícita:
- Crea un *loader* pequeño.
- Convierte el *batch* a **(T, B, C, H, W)**:
  - Si ya viene 5D (B,T,C,H,W), solo permuta a (T,B,C,H,W).
  - Si viene 4D (B,C,H,W), **activa codificación en runtime** (`set_encode_runtime`) y obtiene la secuencia temporal.
- Ejecuta `forward` con **AMP** en CUDA; si falla, reintenta en **FP32**.
- Limpia el modo *runtime encode* al finalizar.

> Esta celda te ayuda a aislar problemas de forma/dispositivo y a validar que la **codificación temporal** está correctamente parametrizada para tu modelo.

[↑ Volver al índice](#toc)


In [6]:
# =============================================================================
# Forward manual: loader -> (T,B,C,H,W) con fallback AMP
# =============================================================================
from contextlib import nullcontext

# 1) Loader pequeño con tu helper
tr, va, te = make_loader_fn(
    task=task_list[0],
    batch_size=8,
    encoder=ENCODER,   # <-- preset
    T=T,               # <-- preset
    gain=GAIN,         # <-- preset
    tfm=tfm,
    seed=SEED,
)
xb, yb = next(iter(tr))
print("batch del loader:", xb.shape, yb.shape)

# 2) A (T,B,C,H,W) con helper (activa runtime encode si hace falta)
used_runtime_encode = False
try:
    x5d, used_runtime_encode = to_5d(xb, ENCODER, T, GAIN, device)
    print("x5d.device:", x5d.device, "| shape:", tuple(x5d.shape))

    # 3) Modelo y forward con fallback AMP
    model = make_model_fn(tfm).to(device).eval()
    use_cuda = torch.cuda.is_available()
    ctx = torch.amp.autocast("cuda", enabled=use_cuda) if use_cuda else nullcontext()

    try:
        with torch.inference_mode(), ctx:
            yhat = model(x5d.to(device, non_blocking=True))
        print("[forward] ejecutado con AMP" if use_cuda else "[forward] ejecutado en FP32")
    except Exception as e:
        if use_cuda:
            print("[forward] AMP falló, reintento en FP32. Motivo:", str(e))
        with torch.inference_mode():
            yhat = model(x5d.to(device, dtype=torch.float32, non_blocking=True))
        print("[forward] ejecutado en FP32")

    print("yhat:", tuple(yhat.shape))

finally:
    # 4) Limpieza del runtime encode (si se usó)
    if used_runtime_encode:
        training.set_encode_runtime(None)


batch del loader: torch.Size([8, 10, 1, 66, 200]) torch.Size([8, 1])
x5d.device: cpu | shape: (10, 8, 1, 66, 200)
[forward] ejecutado con AMP
yhat: (8, 1)


<a id="sec-07"></a>
## 7) Echo de configuración del bench

Imprime los parámetros efectivos del `DataLoader` y del balanceo **online** activos en este cuaderno.  
Sirve para dejar rastro cuando compares *throughput* entre presets o máquinas.

> Recuerda: el **balanceo offline** (por imágenes) se decide **antes** en los cuadernos de *prep*; aquí solo afecta el **balanceo online** en el `DataLoader`.

[↑ Volver al índice](#toc)


In [7]:
# =============================================================================
# Bench: echo de configuración efectiva
# =============================================================================
print_bench_config(
    NUM_WORKERS=NUM_WORKERS, PREFETCH=PREFETCH,
    PIN_MEMORY=PIN_MEMORY, PERSISTENT=PERSISTENT,
    USE_ONLINE_BALANCING=USE_ONLINE_BALANCING,  # ← sin USE_OFFLINE_BALANCED
)


[Bench workers=8 prefetch=2 pin=True persistent=True | online_bal=False


<a id="sec-08"></a>
## 8) Toggle: *it/s* por época

Activa un *hook* que imprime **iteraciones/segundo** (*it/s*) por época durante entrenamiento.  
Es útil cuando luego ejecutes 03 para tener una **lectura rápida de rendimiento**.

- Llama a `enable_epoch_ips()` para **activar**.
- Llama a `disable_epoch_ips()` para **restaurar** el comportamiento normal.

> Este *toggle* no cambia la métrica ni el *logger*; solo añade un contador de *throughput* por época.

[↑ Volver al índice](#toc)


In [8]:
# =============================================================================
# Toggle: imprimir it/s por época durante entrenamiento
# =============================================================================
enable_epoch_ips()
print("it/s por época ACTIVADO. Llama a disable_epoch_ips() para restaurar.")


it/s por época ACTIVADO. Llama a disable_epoch_ips() para restaurar.


<a id="sec-09"></a>
## 9) (Opcional) Micro-bench pipeline *it/s*

Mide un **throughput aproximado** de `loader → modelo` (con AMP si procede) durante un número acotado de iteraciones.

Recomendaciones:
- Usa un `batch_size` realista para tu preset.
- Asegúrate de que el primer *batch* ha calentado cachés (la función ya hace *warmup* básico).
- Interpreta el valor como **tendencia**: variará con el estado del sistema, E/S de disco, *workers* y *augment*.

> Úsalo para comparar **H5 offline** vs **CSV + runtime** y valorar impacto de `num_workers/prefetch` antes de correr entrenos largos.

[↑ Volver al índice](#toc)


In [9]:
# =============================================================================
# (Opcional) Micro-bench pipeline: it/s (loader + modelo)
# =============================================================================
tr, va, te = make_loader_fn(
    task=task_list[0],
    batch_size=8,
    encoder=ENCODER, T=T, gain=GAIN,
    tfm=tfm, seed=SEED,
)

its = pipeline_its(
    model=make_model_fn(tfm).to(device).eval(),
    loader=tr, device=device,
    iters=100, use_amp=True,
    encoder=ENCODER, T=T, gain=GAIN,
)
print(f"pipeline it/s ≈ {its:.2f}")


pipeline it/s ≈ 35.77


<a id="sec-10"></a>
## 10) (Opcional) Smokes de métodos con `run_continual`

Lanza **ejecuciones cortas** (“*smokes*”) por método de aprendizaje continuo usando la **misma configuración** del preset, cambiando solo:
- `continual.method` (p. ej., `naive`, `ewc`, `rehearsal`, `rehearsal+ewc`, `as-snn`, …)
- `continual.params` (los mínimos para que arranque).

¿Qué valida?
- Integración **end-to-end**: *task list* → *loader factory* → modelo → bucle continual.
- Que cada método **enciende** con sus parámetros básicos.
- Que se generan salidas en `outputs/` (manifiestos y métricas mínimas).

> Mantén estos *smokes* **breves** (pocas épocas y/o menos datos) para no sesgar el *bench* ni bloquear la GPU.

[↑ Volver al índice](#toc)


In [10]:
# =============================================================================
# (Opcional) Smokes de métodos con run_continual (cfg del preset)
# =============================================================================
from copy import deepcopy
from src.runner import run_continual

DO_RUNS = True  # pon False para saltar

if DO_RUNS:
    base_cfg = load_preset(ROOT / "configs" / "presets.yaml", PRESET)
    base_cfg = deepcopy(base_cfg)
    base_cfg["data"]["seed"] = SEED  # fija la semilla si quieres

    METHODS = {
        "naive": {},
        "ewc": {"lam": 1e9, "fisher_batches": 200},
        "rehearsal": {"buffer_size": 1000, "replay_ratio": 0.2},
        "rehearsal+ewc": {"buffer_size": 1000, "replay_ratio": 0.2, "lam": 7e8, "fisher_batches": 200},
        "as-snn": {"gamma_ratio": 0.3, "lambda_a": 1.59168, "ema": 0.824},
    }

    METHODS = {
        "sa-snn": {
            "k": 10,
            "tau": 10.0,
            "th_min": 1.0,
            "th_max": 2.0,
            "p": 2000000,
            "vt_scale": 1.0,
            # "attach_to": "classifier.0",   # opcional: si sabes la capa densa
            "flatten_spatial": False,
            "assume_binary_spikes": False,
            "reset_counters_each_task": False,
        },
    }


    out_runs = []
    for mname, mparams in METHODS.items():
        cfg_i = deepcopy(base_cfg)
        cfg_i["continual"]["method"] = mname
        cfg_i["continual"]["params"] = mparams

        print(
            f"\n>>> SMOKE: preset={PRESET} | method={mname} | "
            f"seed={cfg_i['data']['seed']} | enc={cfg_i['data']['encoder']} | kwargs={mparams}"
        )
        out_dir, _ = run_continual(
            task_list=task_list,
            make_loader_fn=make_loader_fn,  # factory de Celda 4
            make_model_fn=make_model_fn,
            tfm=tfm,
            cfg=cfg_i,               # CONFIG COMPLETA del preset con el método cambiado
            preset_name=PRESET,      # solo naming
            out_root=ROOT / "outputs",
            verbose=True,
        )
        out_runs.append(out_dir)
    print("\nHecho:", [str(p) for p in out_runs])



>>> SMOKE: preset=fast | method=sa-snn | seed=42 | enc=rate | kwargs={'k': 10, 'tau': 10.0, 'th_min': 1.0, 'th_max': 2.0, 'p': 2000000, 'vt_scale': 1.0, 'flatten_spatial': False, 'assume_binary_spikes': False, 'reset_counters_each_task': False}

--- Tarea 1/2: circuito1 | preset=fast | method=sa-snn_k10_tau10_th1-2_p2000000 | B=128 T=10 AMP=True | enc=rate ---
[SA-SNN] attach_to=auto -> hook en primera capa Linear: Linear(in_features=1152, out_features=100, bias=True)
[SA-SNN] tracing cada 100 pasos sobre Linear (auto)
[SA-SNN] Linear t=9/9 | trace_mean=2.9082 | score_mean=2.9062 | mask≈10.0% (target≈10.0%) | k=10 N=100 | tau=10 | vt=1 | th∈[1,2]
[SA-SNN] Linear t=9/9 | trace_mean=2.8848 | score_mean=2.8828 | mask≈10.0% (target≈10.0%) | k=10 N=100 | tau=10 | vt=1 | th∈[1,2]
[SA-SNN] Linear t=9/9 | trace_mean=2.8652 | score_mean=2.8633 | mask≈10.0% (target≈10.0%) | k=10 N=100 | tau=10 | vt=1 | th∈[1,2]
[SA-SNN] Linear t=9/9 | trace_mean=2.8008 | score_mean=2.7969 | mask≈10.0% (target≈1